## run this cell before starting spark cluster connection (corrects module error)

In [2]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data

In [5]:
df_ps_final = spark.read.json("/user/sprenner/forest_forward_selection_data.json")

In [6]:
df_ps_final.printSchema()

root
 |-- account: string (nullable = true)
 |-- class_bucket: string (nullable = true)
 |-- clientState: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- datasetScope: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filesize_double: double (nullable = true)
 |-- hostname: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- remoteSite: string (nullable = true)
 |-- scope: string (nullable = true)
 |-- traceId: string (nullable = true)
 |-- traceIp: string (nullable = true)
 |-- traceTimeentry: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- transferEnd: double (nullable = true)
 |-- transferStart: double (nullable = true)
 |-- uuid: string (nullable = true)



In [7]:
df_ps_final.count()

2687516

### Scale Numeric Features

In [8]:
df_ps_final.drop('traceTimeentry')
df_ps_final = df_ps_final.dropna()
df_ps_final.count()

2687516

In [9]:
#df_ps_final.write.format("json").save("/user/sprenner/forward_selection_data.json")

In [10]:
pd.set_option('display.max_columns', None)
df_ps_final.describe().toPandas()

summary  account class_bucket         clientState  \
0   count  2687516      2687516             2687516   
1    mean     None         None                None   
2  stddev     None         None                None   
3     min      aad     cache120                DONE   
4     max   zzheng      nocache  ServiceUnavailable   

                                             dataset datasetScope eventType  \
0                                            2687516      2687516   2687516   
1                                               None         None      None   
2                                               None         None      None   
3                           BBS_M1000_nominal_100418    cond09_mc  download   
4  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...       valid3  download   

  eventVersion                          filename       filesize_double  \
0      2687516                           2687516               2687516   
1         None                              None  2.0072481545558798E8   
2         None                              None   7.736077827603424E8   
3       1.13.2  AOD.05371635._000001.pool.root.1                   5.0   
4          api            wjets_v1.15_small.root       2.7575133067E10   

                    hostname protocol                remoteSite      scope  \
0                    2687516  2687516                   2687516    2687516   
1                       None     None                      None       None   
2                       None     None                      None       None   
3     ReCaS-ui-01.cs.infn.it     davs           AGLT2_CALIBDISK  cond09_mc   
4  xenia2.nevis.columbia.edu      srm  ZA-WITS-CORE_SCRATCHDISK     valid3   

                            traceId                    traceIp  \
0                           2687516                    2687516   
1                          Infinity                       None   
2                               NaN                       None   
3  00000a90a2c841548def8b19b974a7bf  2001:1458:201:e3::100:107   
4  fffffa77425f474b9fa9d590bdc6e23c        ::ffff:93.175.19.27   

               traceTimeentry    traceTimeentryUnix           transferEnd  \
0                     2687516               2687516               2687516   
1                        None  1.5237838542132177E9  1.5237838550567381E9   
2                        None     731543.0885954848     731542.3517505171   
3  2018-03-31T22:00:04.482045   1.522533604482045E9   1.522533601847934E9   
4  2018-04-30T21:59:59.221596   1.525125599221596E9    1.52512559881952E9   

          transferStart                              uuid  
0               2687516                           2687516  
1  1.5237838036445189E9                              None  
2     731543.5454882922                              None  
3   1.522527378084161E9  0000600be6604da0bb3539eec6e3c4ba  
4   1.525125588602087E9  ffffacfbecb248dd8f14a744eaee2a2f

In [11]:
df_ps_final.printSchema()

root
 |-- account: string (nullable = true)
 |-- class_bucket: string (nullable = true)
 |-- clientState: string (nullable = true)
 |-- dataset: string (nullable = true)
 |-- datasetScope: string (nullable = true)
 |-- eventType: string (nullable = true)
 |-- eventVersion: string (nullable = true)
 |-- filename: string (nullable = true)
 |-- filesize_double: double (nullable = true)
 |-- hostname: string (nullable = true)
 |-- protocol: string (nullable = true)
 |-- remoteSite: string (nullable = true)
 |-- scope: string (nullable = true)
 |-- traceId: string (nullable = true)
 |-- traceIp: string (nullable = true)
 |-- traceTimeentry: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- transferEnd: double (nullable = true)
 |-- transferStart: double (nullable = true)
 |-- uuid: string (nullable = true)



In [37]:
categoricalFeatures = ['account', 'clientState', 'dataset', 'datasetScope', 'eventType', 'eventVersion', 'filename', 'hostname', 'protocol', 'remoteSite', 'scope', 'traceId', 'traceIp', 'uuid']

encoded = df_ps_final

for elem in categoricalFeatures:
    encoded = encoded.withColumn(elem + "Hash", hash(encoded[elem]))

In [38]:
encoded = encoded.dropna()
encoded.describe().toPandas()

summary  account class_bucket         clientState  \
0   count  2687516      2687516             2687516   
1    mean     None         None                None   
2  stddev     None         None                None   
3     min      aad     cache120                DONE   
4     max   zzheng      nocache  ServiceUnavailable   

                                             dataset datasetScope eventType  \
0                                            2687516      2687516   2687516   
1                                               None         None      None   
2                                               None         None      None   
3                           BBS_M1000_nominal_100418    cond09_mc  download   
4  valid3.110401.PowhegPythia_P2012_ttbar_nonallh...       valid3  download   

  eventVersion                          filename       filesize_double  \
0      2687516                           2687516               2687516   
1         None                              None  2.0072481545558798E8   
2         None                              None   7.736077827603424E8   
3       1.13.2  AOD.05371635._000001.pool.root.1                   5.0   
4          api            wjets_v1.15_small.root       2.7575133067E10   

                    hostname protocol                remoteSite      scope  \
0                    2687516  2687516                   2687516    2687516   
1                       None     None                      None       None   
2                       None     None                      None       None   
3     ReCaS-ui-01.cs.infn.it     davs           AGLT2_CALIBDISK  cond09_mc   
4  xenia2.nevis.columbia.edu      srm  ZA-WITS-CORE_SCRATCHDISK     valid3   

                            traceId                    traceIp  \
0                           2687516                    2687516   
1                          Infinity                       None   
2                               NaN                       None   
3  00000a90a2c841548def8b19b974a7bf  2001:1458:201:e3::100:107   
4  fffffa77425f474b9fa9d590bdc6e23c        ::ffff:93.175.19.27   

               traceTimeentry   traceTimeentryUnix          transferEnd  \
0                     2687516              2687516              2687516   
1                        None  1.523783854213218E9  1.523783855056741E9   
2                        None    731543.0885954745    731542.3517492709   
3  2018-03-31T22:00:04.482045  1.522533604482045E9  1.522533601847934E9   
4  2018-04-30T21:59:59.221596  1.525125599221596E9   1.52512559881952E9   

         transferStart                              uuid  \
0              2687516                           2687516   
1  1.523783803644517E9                              None   
2    731543.5454884385                              None   
3  1.522527378084161E9  0000600be6604da0bb3539eec6e3c4ba   
4  1.525125588602087E9  ffffacfbecb248dd8f14a744eaee2a2f   

            accountHash       clientStateHash            datasetHash  \
0               2687516               2687516                2687516   
1  -8.253595791931956E7  -8.788526013591328E8  -4.2551458652869046E7   
2  1.2133826449540486E9   1.452912369272028E7   1.2555896226953044E9   
3           -2144719344           -1394468120            -2147475923   
4            2143592175            1269532611             2147475364   

        datasetScopeHash         eventTypeHash       eventVersionHash  \
0                2687516               2687516                2687516   
1  -4.0535974099238926E8            5.937808E7  -1.9560425947482958E9   
2   1.1133356349716985E9  6.817316567841185E-9    4.037673965503464E8   
3            -2145215727              59378080            -2044360925   
4             2131974379              59378080             1261702107   

           filenameHash          hostnameHash         protocolHash  \
0               2687516               2687516              2687516   
1    -655332.4981655923   8.664258461275803E7  5.390332053252

In [39]:
encoded = encoded.drop('account', 'clientState', 'dataset', 'datasetScope', 'eventType', 'eventVersion', 'filename', 'hostname', 'protocol', 'remoteSite', 'scope', 'traceId', 'traceIp', 'uuid')

In [45]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="class_bucket", outputCol="class")
indexed = indexer.fit(encoded).transform(encoded)

In [46]:
encoded.printSchema()

root
 |-- class_bucket: string (nullable = true)
 |-- filesize_double: double (nullable = true)
 |-- traceTimeentry: string (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- transferEnd: double (nullable = true)
 |-- transferStart: double (nullable = true)
 |-- accountHash: integer (nullable = false)
 |-- clientStateHash: integer (nullable = false)
 |-- datasetHash: integer (nullable = false)
 |-- datasetScopeHash: integer (nullable = false)
 |-- eventTypeHash: integer (nullable = false)
 |-- eventVersionHash: integer (nullable = false)
 |-- filenameHash: integer (nullable = false)
 |-- hostnameHash: integer (nullable = false)
 |-- protocolHash: integer (nullable = false)
 |-- remoteSiteHash: integer (nullable = false)
 |-- scopeHash: integer (nullable = false)
 |-- traceIdHash: integer (nullable = false)
 |-- traceIpHash: integer (nullable = false)
 |-- uuidHash: integer (nullable = false)



In [47]:
encoded.describe().toPandas()

summary class_bucket       filesize_double              traceTimeentry  \
0   count      2687516               2687516                     2687516   
1    mean         None  2.0072481545558798E8                        None   
2  stddev         None   7.736077827603416E8                        None   
3     min     cache120                   5.0  2018-03-31T22:00:04.482045   
4     max      nocache       2.7575133067E10  2018-04-30T21:59:59.221596   

     traceTimeentryUnix           transferEnd        transferStart  \
0               2687516               2687516              2687516   
1  1.5237838542132156E9  1.5237838550567398E9  1.523783803644516E9   
2     731543.0885956176      731542.351749878    731543.5454884754   
3   1.522533604482045E9   1.522533601847934E9  1.522527378084161E9   
4   1.525125599221596E9    1.52512559881952E9  1.525125588602087E9   

            accountHash       clientStateHash            datasetHash  \
0               2687516               2687516                2687516   
1  -8.253595791931956E7  -8.788526013591328E8  -4.2551458652869046E7   
2  1.2133826449540477E9  1.4529123692720124E7   1.2555896226953042E9   
3           -2144719344           -1394468120            -2147475923   
4            2143592175            1269532611             2147475364   

        datasetScopeHash eventTypeHash       eventVersionHash  \
0                2687516       2687516                2687516   
1  -4.0535974099238926E8    5.937808E7  -1.9560425947482958E9   
2    1.113335634971699E9           0.0    4.037673965503459E8   
3            -2145215727      59378080            -2044360925   
4             2131974379      59378080             1261702107   

           filenameHash          hostnameHash         protocolHash  \
0               2687516               2687516              2687516   
1    -655332.4981655923   8.664258461275803E7  5.390332053252491E8   
2  1.2399230843915145E9  1.3238179779357328E9  8.742875323052543E8   
3           -2147481484           -2142788649          -1364943180   
4            2147479105            2143857082           2128034789   

         remoteSiteHash              scopeHash          traceIdHash  \
0               2687516                2687516              2687516   
1  5.8613363583594665E7  -4.1522494967299616E8   -303264.6469312183   
2  1.2996618172877827E9   1.1115090831357343E9  1.239941762293332E9   
3           -2135244854            -2145215727          -2147483615   
4            2141740830             2131974379           2147482933   

            traceIpHash               uuidHash  
0               2687516                2687516  
1   -2.21299552888004E8  -3.1038448530438516E7  
2  1.2157507571173625E9   1.2098174722417417E9  
3           -2143850866            -2147474484  
4            2146432649             2147466273

In [48]:
indexed.describe().toPandas()

summary class_bucket       filesize_double              traceTimeentry  \
0   count      2687516               2687516                     2687516   
1    mean         None  2.0072481545558798E8                        None   
2  stddev         None   7.736077827603425E8                        None   
3     min     cache120                   5.0  2018-03-31T22:00:04.482045   
4     max      nocache       2.7575133067E10  2018-04-30T21:59:59.221596   

    traceTimeentryUnix           transferEnd         transferStart  \
0              2687516               2687516               2687516   
1  1.523783854213218E9  1.5237838550567403E9  1.5237838036445167E9   
2     731543.088595469     731542.3517492627     731543.5454884416   
3  1.522533604482045E9   1.522533601847934E9   1.522527378084161E9   
4  1.525125599221596E9    1.52512559881952E9   1.525125588602087E9   

            accountHash       clientStateHash            datasetHash  \
0               2687516               2687516                2687516   
1  -8.253595791931956E7  -8.788526013591328E8  -4.2551458652869046E7   
2  1.2133826449540486E9   1.452912369272028E7   1.2555896226953046E9   
3           -2144719344           -1394468120            -2147475923   
4            2143592175            1269532611             2147475364   

        datasetScopeHash eventTypeHash       eventVersionHash  \
0                2687516       2687516                2687516   
1  -4.0535974099238926E8    5.937808E7  -1.9560425947482958E9   
2   1.1133356349716985E9           0.0    4.037673965503463E8   
3            -2145215727      59378080            -2044360925   
4             2131974379      59378080             1261702107   

          filenameHash          hostnameHash         protocolHash  \
0              2687516               2687516              2687516   
1   -655332.4981655923   8.664258461275803E7  5.390332053252491E8   
2  1.239923084391515E9  1.3238179779357333E9   8.74287532305254E8   
3          -2147481484           -2142788649          -1364943180   
4           2147479105            2143857082           2128034789   

         remoteSiteHash              scopeHash           traceIdHash  \
0               2687516                2687516               2687516   
1  5.8613363583594665E7  -4.1522494967299616E8    -303264.6469312183   
2  1.2996618172877853E9   1.1115090831357331E9  1.2399417622933323E9   
3           -2135244854            -2145215727           -2147483615   
4            2141740830             2131974379            2147482933   

            traceIpHash               uuidHash               class  
0               2687516                2687516             2687516  
1   -2.21299552888004E8  -3.1038448530438516E7   2.546516560273502  
2  1.2157507571173606E9   1.2098174722417407E9  2.4244943548508395  
3           -2143850866            -2147474484                 0.0  
4            2146432649             2147466273                 8.0

In [50]:
indexed.write.format("json").save("/user/sprenner/forest_forward_selection_data_indexed_hashed.json")

In [ ]:
#data =  spark.read.json("/user/sprenner/forest_forward_selection_data_indexed_hashed.json")

In [49]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator




assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash","eventVersionHash","filenameHash", \
                                       "filesize_double","hostnameHash","protocolHash","remoteSiteHash","scopeHash","traceIdHash","traceIpHash","traceTimeentryUnix", \
                                       "transferEnd","transferStart","uuidHash"], outputCol="features")
output = assembler.transform(data)
output = output.select("class", "features")
        
train_data, test_data = output.randomSplit([.8,.2],seed=1234)
rf = RandomForestClassifier(labelCol='class', featuresCol='features',numTrees=100)
model = rf.fit(train_data)
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

rfModel = model.stages[2]
print(rfModel)  # summary only

Test Error = 0.547743


AttributeError: 'RandomForestClassificationModel' object has no attribute 'stages'

In [30]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import copy


# starts forward selection
def forwardSelection(data, features, target):
    featureList = copy.deepcopy(features)
    selected = []
    #keep track what the current error is to minimize it
    referenceScore = None
    while len(featureList) > 0:
        best = getBestFeature(data, featureList, selected, target, referenceScore)
        if best is not None:
            referenceScore = best[1]
            selected.append(best[0])
            featureList.remove(best[0])
            print("Selected " + str(best[0]))
            print("Current Feature Set: " + str(selected))
        else:
            print("No further improvement with removing features, stopping selection.")
            break
    return selected

# selects feature which yields the largest improvement together with already selected features (include)    
def getBestFeature(data, features, included, target, referenceScore):
    print("Getting next best feature...")
    if len(features) == 0:
        return "No features passed."
    include = copy.deepcopy(included)
    bestFeature = None
    for feature in features:
        # Don't include target feature, would be the best predictor otherwise
        if feature == target:
            print("Skipping " + str(feature))
            continue
        print("Calculating score for column " + str(feature) + " including " + str(include))
        # temporarily add current feature to feature set
        include.append(feature)
        assembler = VectorAssembler(inputCols=include, outputCol="features")
        output = assembler.transform(data)
        output = output.select(target, "features")
        train_data, test_data = output.randomSplit([.8,.2],seed=1234)
        error = regressionSquaredError(target, train_data)
        # first iteration
        if bestFeature is None:
            # first iteration overall
            if referenceScore is None:
                print("First feature overall found: " + str(feature) + " with score " + str(error))
                bestFeature = (feature, error)
                # for first iteration overall referenceScore must be set here
                referenceScore = error
            # first iteration but already reference error available
            elif error < referenceScore:
                print("First feature for loop found: " + str(feature) + " with score " + str(error))
                bestFeature = (feature, error)
        # the error must be smaller than previous selection and smaller than with fewer features
        elif error < bestFeature[1] and error < referenceScore:
            bestFeature = (feature, error)
            print("Better feature found: " + str(feature) + " with score " + str(error))
        # remove temporary feature from feature set to start again with next feature
        include.remove(feature)
    return bestFeature
        
# performs linear regression and returns mean squared error      
def regressionSquaredError(label, data):
    lr = LinearRegression(labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    linearModel = lr.fit(data)
    return linearModel.summary.rootMeanSquaredError

# performs linear regression and returns r2        
def regressionR2(label, data):
    lr = LinearRegression(labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    linearModel = lr.fit(data)
    return linearModel.summary.r2

In [31]:
df_ps_final.schema.names

['accountHash',
 'clientStateHash',
 'datasetHash',
 'datasetScopeHash',
 'dataset_versionHash',
 'datatypeHash',
 'eventTypeHash',
 'eventVersionHash',
 'filenameHash',
 'filesize_double',
 'hits',
 'hostnameHash',
 'prod_stepHash',
 'projectHash',
 'protocolHash',
 'remoteSiteHash',
 'run_numberHash',
 'scopeHash',
 'stream_nameHash',
 'traceIdHash',
 'traceIpHash',
 'traceTimeentryUnix',
 'transferEnd',
 'transferStart',
 'uuidHash']

### Perform forward selection

In [32]:
selected_features = forwardSelection(df_ps_final, df_ps_final.schema.names, "hits")

Getting next best feature...
Calculating score for column accountHash including []
First feature overall found: accountHash with score 10279.0082429
Calculating score for column clientStateHash including []
Calculating score for column datasetHash including []
Better feature found: datasetHash with score 10265.8577577
Calculating score for column datasetScopeHash including []
Better feature found: datasetScopeHash with score 10005.9593602
Calculating score for column dataset_versionHash including []
Calculating score for column datatypeHash including []
Calculating score for column eventTypeHash including []
Calculating score for column eventVersionHash including []
Calculating score for column filenameHash including []
Calculating score for column filesize_double including []
Skipping hits
Calculating score for column hostnameHash including []
Calculating score for column prod_stepHash including []
Better feature found: prod_stepHash with score 9596.0582614
Calculating score for colum

Calculating score for column uuidHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash']
Better feature found: uuidHash with score 8863.31870073
Selected uuidHash
Current Feature Set: ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Getting next best feature...
Calculating score for column accountHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
First feature for loop found: accountHash with score 8825.20314066
Calculating score for column clientStateHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Calculating score for column datasetHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Calculating score for column datasetScopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Calculating score for column datatypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Better feature found: datatypeHa

Calculating score for column run_numberHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column scopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column stream_nameHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column traceIdHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column transferEnd including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column transferStart including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Selected accountHash
Current Feature Set: ['prod_st

Calculating score for column datasetScopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
First feature for loop found: datasetScopeHash with score 8391.96017081
Calculating score for column datatypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
Better feature found: datatypeHash with score 8384.9034658
Calculating score for column eventTypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
Calculating score for column eventVersionHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
Calculating score for column filenam

Calculating score for column filesize_double including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash']
Better feature found: filesize_double with score 8366.39187002
Skipping hits
Calculating score for column protocolHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash']
Better feature found: protocolHash with score 8366.09341767
Calculating score for column remoteSiteHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash']
Calculating score for column scopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnam

Calculating score for column transferEnd including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash']
Calculating score for column transferStart including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash']
Selected stream_nameHash
Current Feature Set: ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash', 'stream_nameHash']
Getting next best feature...
Calculating score for column clientStateHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 

Calculating score for column eventTypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash', 'stream_nameHash', 'eventVersionHash', 'filesize_double']
Calculating score for column filenameHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash', 'stream_nameHash', 'eventVersionHash', 'filesize_double']
First feature for loop found: filenameHash with score 8335.43291646
Skipping hits
Calculating score for column scopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSi

In [33]:
selected_features

['prod_stepHash',
 'traceIpHash',
 'dataset_versionHash',
 'uuidHash',
 'traceTimeentryUnix',
 'hostnameHash',
 'accountHash',
 'run_numberHash',
 'datasetHash',
 'datatypeHash',
 'projectHash',
 'protocolHash',
 'remoteSiteHash',
 'stream_nameHash',
 'eventVersionHash',
 'filesize_double',
 'traceIdHash',
 'filenameHash']